In [1]:
import os
import SimpleITK as sitk
import numpy as np
import pandas as pd
from tests.liver_seg_to_roi import \
    get_roi_array_from_seg_volume, save_array_to_nii, get_image_path_by_id

In [2]:
df_init = pd.DataFrame()

img_dir =  'nnUNet_raw_data_base/nnUNet_test_data/test_img_in_nii/'
seg_dir =  'nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/'
targ_dir = 'nnUNet_raw_data_base/nnUNet_seg_to_AI-ROI/roi_in_nii/'

csv_write_path = 'csv/AI_ROI_attenuation_using_AI_seg.csv'

label_paths = []
for r, d, f in os.walk(seg_dir):
    for file in f:
        if '.nii.gz' in file :
            label_paths.append(os.path.join(r, file))
#             print(label_path)
            
for idx, label_path in enumerate(sorted(label_paths)):
    
    d = label_path.split('/')[-1]             
    pid = d[:14]
    seg_path = label_path
    img_path = get_image_path_by_id(pid, img_dir)
    roi_path = targ_dir + pid + '_AI_ROI_seg.nii.gz'
    
    ct_sitk = sitk.ReadImage(img_path)
    ct_array =  sitk.GetArrayFromImage(ct_sitk)
    
    seg_sitk = sitk.ReadImage(seg_path)
    seg_array =  sitk.GetArrayFromImage(seg_sitk)    

    liver_seged = np.multiply((seg_array>0)*1.0,ct_array)
    density_3d =  np.sum(liver_seged)/ np.sum(seg_array >0)
            
    roi_array = get_roi_array_from_seg_volume(seg_path, roi_path)
    roi_of_liver = np.multiply((roi_array>0)*1.0,ct_array)
    density_roi =  np.sum(roi_of_liver)/ np.sum(roi_array >0)    

    save_array_to_nii(roi_array, seg_path, roi_path)
    print(idx, img_path, '\n', roi_path, '\n')
    
    df = pd.DataFrame({'CT_id': pid,    '3D_attenuation_using_seg':round(density_3d,5),                   
                'ROI_attenuation_using_seg':round(density_roi,5)},index=[0])   
    df_init = df_init.append(df)
    df_init.to_csv( csv_write_path )
    print(pid, seg_path,'\n',  roi_path,'\n\n',  img_path, '\n\n\n\n') 


slice_list [13, 15, 17]
nnUNet_raw_data_base/nnUNet_seg_to_AI-ROI/roi_in_nii/LIDC-IDRI_0002_AI_ROI_seg.nii.gz saved
0 nnUNet_raw_data_base/nnUNet_test_data/test_img_in_nii/LIDC-IDRI_0002_img_nnunet_to_seg_0000.nii.gz 
 nnUNet_raw_data_base/nnUNet_seg_to_AI-ROI/roi_in_nii/LIDC-IDRI_0002_AI_ROI_seg.nii.gz 

LIDC-IDRI_0002 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/LIDC-IDRI_0002_img_nnunet_to_seg_top1mask.nii.gz 
 nnUNet_raw_data_base/nnUNet_seg_to_AI-ROI/roi_in_nii/LIDC-IDRI_0002_AI_ROI_seg.nii.gz 

 nnUNet_raw_data_base/nnUNet_test_data/test_img_in_nii/LIDC-IDRI_0002_img_nnunet_to_seg_0000.nii.gz 






In [3]:
import os, glob
from tests.jpack_generate_roi import seg_in_roi_by_screenshots

img_dir =  'nnUNet_raw_data_base/nnUNet_test_data/test_img_in_nii/'
roi_dir =  'nnUNet_raw_data_base/nnUNet_seg_to_AI-ROI/roi_in_nii/'
screenshots_dir = 'nnUNet_raw_data_base/nnUNet_seg_to_AI-ROI/roi_in_png/'

for idx,d in enumerate(sorted(os.listdir(roi_dir))): 
    if '.nii.gz' in d:           
        indices = [i for i, c in enumerate(d) if c == '_']
        patient_id = d[:indices[1]]
        print(d, patient_id)
        
        ct_image_path = glob.glob(os.path.join(img_dir, '*'+patient_id +'*'))[0]
        roi_path =glob.glob(os.path.join(roi_dir, '*'+patient_id +'*'))[0]
        print(idx+1,patient_id, ct_image_path.split('/')[-1], roi_path.split('/')[-1])   
        seg_in_roi_by_screenshots(ct_image_path,roi_path,screenshots_dir,patient_id )
        print(idx+1,patient_id,' done \n\n\n\n\n')

LIDC-IDRI_0002_AI_ROI_seg.nii.gz LIDC-IDRI_0002
1 LIDC-IDRI_0002 LIDC-IDRI_0002_img_nnunet_to_seg_0000.nii.gz LIDC-IDRI_0002_AI_ROI_seg.nii.gz
im_array.shape[0] 130 auto_seg_array.shape[0] 130
roi_slice_list [13, 15, 17]
1 LIDC-IDRI_0002  done 





